In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import operator
import re
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from collections import Counter
from sklearn import linear_model
import csv
from datetime import datetime

%matplotlib inline

In [2]:
df1=pd.read_csv("../data/train_1.csv")

In [3]:
matrix = df1.loc[:, '2015-11-01':'2016-03-01'].values
matrix_remove_2_29 = np.delete(matrix, 120, axis=1) # 2017 does not have Feb 29th
print(matrix_remove_2_29.shape)

(145063, 121)


In [4]:
delete_nan_row = {}
z = np.where(np.isnan(matrix_remove_2_29))
print(z[0])
for x in z[0]:
    if x in delete_nan_row:
        delete_nan_row[x] += 1
    else:
        delete_nan_row[x] = 1
hp_threshold = 60 # delete row if its NaN larger than this
idx = [k for k, v in delete_nan_row.iteritems() if v > hp_threshold]
print(len(idx))
not_much_nan_m = np.delete(matrix_remove_2_29, idx, axis=0)
print(not_much_nan_m.shape)

[     4      4      4 ..., 145062 145062 145062]
13785
(131278, 121)


In [5]:
# Imputation for missing value
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(not_much_nan_m)
#Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
no_nan_m = imp.transform(not_much_nan_m)    
np.where(np.isnan(no_nan_m))

(array([], dtype=int64), array([], dtype=int64))

In [6]:
#multitask Lasso
scaled_X_2 = preprocessing.scale(no_nan_m[:, 0:61])
y2 = no_nan_m[:, 61:]
clf = linear_model.MultiTaskLasso(alpha=0.1, max_iter = 3000)
clf.fit(scaled_X_2, y2)

/Applications/anaconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:1687: UserWarning: Objective did not converge, you might want to increase the number of iterations
  warnings.warn('Objective did not converge, you might want'


MultiTaskLasso(alpha=0.3, copy_X=True, fit_intercept=True, max_iter=3000,
        normalize=False, random_state=None, selection='cyclic', tol=0.0001,
        warm_start=False)

In [8]:
predict_X_raw = df1.loc[:, '2016-11-01':'2016-12-31'].values
print(predict_X_raw.shape)
imp2 = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp2.fit(predict_X_raw)
predict_X_tf = imp2.transform(predict_X_raw)
predict_X = preprocessing.scale(predict_X_tf)
y_multi_lasso = clf.predict(predict_X)

(145063, 61)


In [10]:
print(y_multi_lasso.shape)

(145063, 59)


In [12]:
# Lasso regression
scaled_X = preprocessing.scale(no_nan_m[:, 0:61])
y = no_nan_m[:, 61]
reg = linear_model.Lasso(alpha = 0.1, max_iter = 5000)
reg.fit(scaled_X, y)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=5000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [22]:
y_one = reg.predict(predict_X)
print(y_one.shape)
y_one = np.reshape(y_one, (145063, 1))
print(y_one.shape)

(145063,)
(145063, 1)


In [23]:
print(type(y_multi_lasso))
print(type(y_one))
final_y = np.concatenate((y_one, y_multi_lasso), axis=1)

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [24]:
idx_less_zero = np.where(final_y < 0)
final_y[idx_less_zero] = 0

In [26]:
print(final_y.shape)
np.savetxt("../tmp/pre_submission_07281746.csv", final_y, delimiter=",")

(145063, 60)


In [39]:
import sys
sys.path.insert(0, '../')

from utils import generate_submission_1

In [ ]:
result = final_y

map_pos = pd.read_csv('../tmp/map_pos.csv').values

res_dict = {}
days = 60
page_num = 145063
for i in range(page_num * days):
    res_dict[map_pos[i, 2]] = temp[map_pos[i, 0], map_pos[i, 1]]
ttime = datetime.now()
time2str = strc(ttime.month) + strc(ttime.day) + strc(ttime.hour) + strc(ttime.minute)
filename = '../result/submission_' + time2str + '.csv'




In [ ]:
submission_result = pd.DataFrame(res_dict, columns = ['Id', 'Visits'])
submission_result.to_csv(filename, index=False)

In [55]:
def strc(x):
    s = str(x) if x >= 10 else '0' + str(x)
    return s

In [64]:
ttime = datetime.now()
time2str = strc(ttime.month) + strc(ttime.day) + strc(ttime.hour) + strc(ttime.minute)
filename = '../result/submission_' + time2str + '.csv'
days = 60
page_num = 145063
submission_result = pd.read_csv('../data/sample_submission_1.csv')
sub_res = submission_result.values
map_pos = pd.read_csv('../tmp/map_pos.csv').values
with open(filename, 'wb') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Id', 'Visits'])
    for i in range(page_num * days):
        value = result[map_pos[i, 0], map_pos[i, 1]]
        writer.writerow([sub_res[i, 0], value])

In [62]:
print(submission_result.shape)
sub_res = submission_result.values

(8703780, 2)


In [65]:
new_res = pd.read_csv(filename).values
print(new_res.shape)

(8703780, 2)
